## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
from numpy import linalg as LA
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [9]:
n      = 1.33
c      = 299792458/n
R      = 7.3/2 #--------------------- radius of tank
v      = n * c
times  = linspace(7e-10,1e-7,200000)

c1       = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
c2       = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
c3       = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
c4       = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
xA,xB,h  = R-3.0, -(R-6.5), 1.3

plotme = 'b vs phi'                                                                                         #------PARAM

A   = array( [xA,  sqrt(R**2-xA**2), h] )
B   = array( [xB, -sqrt(R**2-xB**2), h] )
AB  = B-A
nAB = LA.norm(AB)
print(nAB)
den = c*c - v*v
print('A : ', A)
print('B : ', B)

6.270604132263064
A :  [0.65     3.591657 1.3     ]
B :  [ 2.85       -2.28035085  1.3       ]


In [ ]:
def exit_brightness(nAB,BD,c,v,L,alpha):
    x     = nAB
    k     = LA.norm(BD)
    t1    = x/v
    t2    = k/c
    tt    = t1 + t2    
    x1    = (c*c*tt*v-L*v*v*cos(alpha)+sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den
    v1    = ( (c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den
    k1    = sqrt(L*L + x1*x1 - 2*L*x1*cos(alpha))
    beta1 = pi - arccos((x1*x1 + k1*k1 - L*L)/(2*x1*k1))
    v1t   = v1*sin(beta1)
    omega1 = v1t / k1
    b1 = abs(omega1 / (k1**2))
    return b1

In [10]:
%matplotlib
plt.figure()

detector_coordinates = [c1,c2,c3,c4]
colors    = ['r','b','g','c']
detectors = [1,2,3,4]

plotme    = 'b vs t'

for D, color, detector in zip(detector_coordinates, colors, detectors):
    
    AD    = D - A
    BD    = D - B
    L     = LA.norm(AD)
    alpha = arccos((sum(AD*AB))/(L*nAB)) # OR use alpha1 = arccos( (L*L+x*x-k*k)/(2*x*L) )
    
    T, aX1, aX2, aB1, aB2, BETA1, BETA2 = [], [], [], [], [], [], []
    
    
    num_xc = L*cos(alpha)*(c*c-v*v) + sqrt(-c**4*L**2*sin(alpha)**2 + c**2*L**2*v**2*sin(alpha)**2)
    den    = (c**2-v**2)
    xc     = (num_xc/den)
    print('Critical distance (from entry point) : {} m'.format(xc))
    #if (xc>nAB) or (xc<0):
    #    print('Sorry! First RID is outside the tank. Not processing this case further.')
        #continue
    #ex_bright = exit_brightness(nAB,BD,c,v,L,alpha)
    for iii,t in enumerate(times):

        x     = v*t
        ratio = x/nAB
        X     = array([ (1-ratio)*A[0] + ratio*B[0] , (1-ratio)*A[1] + ratio*B[1] , h   ])
        AX    = X - A
        DX    = X - D
        k     = LA.norm(DX)

        t1    = t
        t2    = k/c
        tt    = t1 + t2    
        
        x1    = (c*c*tt*v-L*v*v*cos(alpha)+sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den
        x2    = (c*c*tt*v-L*v*v*cos(alpha)-sqrt(v*v*( -L*L*v*v + c*c*L*L + c*c*tt*tt*v*v - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2))) / den

        v1    = ( (c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den
        v2    = ((-c*c*v) + ( (c*c*v**3*(tt*v-L*cos(alpha)))  / (sqrt( v*v*(-L*L*v*v + c*c*(L*L+tt*tt*v*v) - 2*c*c*L*tt*v*cos(alpha) + L*L*v*v*cos(alpha)**2) )) )  ) / den 

        k1    = sqrt(L*L + x1*x1 - 2*L*x1*cos(alpha))
        k2    = sqrt(L*L + x2*x2 - 2*L*x2*cos(alpha))
        
        beta1 = pi - arccos((x1*x1 + k1*k1 - L*L)/(2*x1*k1))
        beta2 = pi - arccos((x2*x2 + k2*k2 - L*L)/(2*x2*k2))
        
        v1t   = v1*sin(beta1)
        v2t   = v2*sin(beta2)

        omega1 = v1t / k1
        omega2 = v2t / k2

        b1 = abs(omega1 / (k1**2))
        b2 = abs(omega2 / (k2**2))

        aX1.append(x1)
        aX2.append(x2)
        aB1.append(b1)
        aB2.append(b2)
        BETA1.append(rad2deg(beta1))
        BETA2.append(rad2deg(beta2))
        T.append(tt)

    aX1, aX2, aB1, aB2, T, BETA1, BETA2 = array(aX1), array(aX2), array(aB1), array(aB2), array(T), array(BETA1), array(BETA2)
    Tn  = T * 1e+9
    TT  = (Tn-Tn.min())
    condition = (aX1>=0) & (aX2<=nAB)
    plt.plot(aX1,aB1,c=color)
    plt.plot(aX2,aB2,c=color)
plt.show()

Using matplotlib backend: Qt5Agg
Critical distance (from entry point) : 0.5391523328682734 m
Critical distance (from entry point) : -1.5771746543902838 m
Critical distance (from entry point) : 0.2692810128961523 m
Critical distance (from entry point) : 2.293678679696532 m


In [5]:
a=[]
b=c=d=a

In [ ]:
    if ptype==('y' or 'Y'):
        print(ex_bright)
        aB1 = aB1/ex_bright
        aB2 = aB2/ex_bright
    
    if plotme == 'x vs t':
        plt.plot(TT[condition], aX1[condition], c=color, ls='--')   # 1 == towards A    2 == towards B
        plt.plot(TT[condition], aX2[condition], c=color, ls='-' , label='Detector {}'.format(detector))
        plt.axhline(xc, c='k', ls=':')
        plt.text(35, round(xc,2)+0.085, r'$x_c = {} \; m$'.format(round(xc,2)), fontsize=16)
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'image distance from the entry point $x_{\pm}$ (m)'       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(0,nAB)
        plt.xlim(0,40)
    elif plotme == 'b vs t':
        plt.plot(TT[condition], aB1[condition], c=color, ls='--')
        plt.plot(TT[condition], aB2[condition], c=color, ls='-' , label=r'Detector {}'.format(detector))
        #plt.axhline(1, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{ground}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
    elif plotme == 'b vs phi':
        plt.plot(BETA1[condition], aB1[condition], c=color, ls='--')
        plt.plot(BETA2[condition], aB2[condition], c=color, ls='-' , label=r'Detector {}'.format(detector))
        #plt.axhline(1, c='k', ls=':')
        plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{ground}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        plt.xlim(0,90)
        #plt.xscale('log')
        #plt.ylim(0,h)
    elif plotme == 'phi vs t':
        plt.plot(TT[condition], BETA1[condition], c=color, ls='--')
        plt.plot(TT[condition], BETA2[condition], c=color, ls='-' , label=r'Detector {}'.format(detector))
        #plt.axhline(1, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(0,90)
        #plt.xlim(0,25)
        #plt.yscale('log')
    #del condition, ex_bright, aB1,aB2,aX1,aX2,BETA1,BETA2,TT,
plt.legend()
plt.show()